In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV 
from sklearn.model_selection import KFold
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import accuracy_score
from keras.layers import Dense
from keras.models import Sequential
from keras.optimizers import SGD
from matplotlib import pyplot as plt
import matplotlib as mpl
import seaborn as sns
import numpy as np
import pandas as pd
import category_encoders as ce
import os
import pickle
import gc
from tqdm import tqdm
import pickle
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.neighbors import KNeighborsRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn import ensemble
import xgboost as xgb
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [3]:
def encode_text_features(encode_decode, data_frame, encoder_isa=None, encoder_mem_type=None):
    # Implement Categorical OneHot encoding for ISA and mem-type
    if encode_decode == 'encode':
        encoder_isa = ce.one_hot.OneHotEncoder(cols=['isa'])
        encoder_mem_type = ce.one_hot.OneHotEncoder(cols=['mem-type'])
        encoder_isa.fit(data_frame, verbose=1)
        df_new1 = encoder_isa.transform(data_frame)
        encoder_mem_type.fit(df_new1, verbose=1)
        df_new = encoder_mem_type.transform(df_new1)
        encoded_data_frame = df_new
    else:
        df_new1 = encoder_isa.transform(data_frame)
        df_new = encoder_mem_type.transform(df_new1)
        encoded_data_frame = df_new
        
    return encoded_data_frame, encoder_isa, encoder_mem_type

In [4]:
def absolute_percentage_error(Y_test, Y_pred):
    error = 0
    for i in range(len(Y_test)):
        if(Y_test[i]!= 0 ):
            error = error + (abs(Y_test[i] - Y_pred[i]))/Y_test[i]
        
    error = error/ len(Y_test)
    return error

In [5]:
def create_model(input_dim):
    # define the keras model
    # DNN 1
    dnn = Sequential()
    dnn.add(Dense(64, input_dim = input_dim, activation='relu'))
    # dnn.add(Dropout(0.1))
    dnn.add(Dense(64, activation='relu'))
    # dnn.add(Dropout(0.1))
    dnn.add(Dense(64, activation='relu'))
    # dnn.add(Dropout(0.1))
    dnn.add(Dense(2, activation='linear'))
    # print('Model : DNN 2', dnn.summary())
    # plot_model(dnn, to_file = 'dnn.png', show_shapes = True)
    return dnn

In [13]:
def process_all(dataset_path, dataset_name, path_for_saving_data):
    
    ################## Data Preprocessing ######################
    df = pd.read_csv(dataset_path)
    encoded_data_frame, encoder_isa, encoder_mem_type = encode_text_features('encode', df, 
                                                                             encoder_isa = None, encoder_mem_type=None)
    total_data = encoded_data_frame.drop(columns = ['arch'])
    total_data = total_data.fillna(0)
    X = total_data.drop(columns = ['runtime','power']).to_numpy()
    # X = total_data.drop(columns = ['runtime']).to_numpy()
    Y = total_data[['runtime', 'power']].to_numpy()
    print('Data X and Y shape', X.shape, Y.shape)
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)
    print('Train Test Split:', X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

    ################## Data Preprocessing ######################


    
    k = 0
    df = pd.DataFrame(columns = ['model_name', 'dataset_name', 'r2', 'mse', 'mape_runtime','mape_power', 'mae' ])
    print('####################################################################')
    r2_scores = []
    mse_scores = []
    mape_runtime_scores = []
    mape_power_scores = []
    mae_scores = []

    cv = ShuffleSplit(n_splits = 10, random_state = 0, test_size = 0.2)

    fold = 1
    for train_index, test_index in cv.split(X):
        model_orig = create_model(input_dim = X.shape[1])

        X_train_fold, X_test_fold, Y_train_fold, Y_test_fold = X[train_index], X[test_index], Y[train_index], Y[test_index]
        scaler_x = StandardScaler()
        X_train_fold = scaler_x.fit_transform(X_train_fold)
        # X_train = scaler_x.fit_transform(X_train)
        X_test_fold = scaler_x.fit_transform(X_test_fold)
        scaler_y = StandardScaler()
        # Y = scaler_y.fit_transform(Y)
        Y_train_fold = scaler_y.fit_transform(Y_train_fold)
        Y_test_fold = scaler_y.fit_transform(Y_test_fold)
        model_orig.compile(loss='mae', optimizer='adam', metrics=['mae'])
        model_orig.fit(X_train_fold, Y_train_fold, epochs = 100, batch_size = 5, verbose = 0)
        Y_pred_fold = model_orig.predict(X_test_fold)

        # save the folds to disk
        data = [X_train_fold, X_test_fold, Y_train_fold, Y_test_fold]
        filename = path_for_saving_data + '/folds_data/' +'dnn' +'_'+ str(fold) + '.pickle'
        pickle.dump(data, open(filename, 'wb'))


        # save the model to disk
        # serialize model to JSON
        filename_1 = path_for_saving_data + '/models_data/' + 'dnn' + '_' + str(fold) + '.json'
        filename_2 = path_for_saving_data + '/models_data/' + 'dnn' + '_' + str(fold) + '.h5'
        fold = fold + 1
        model_json = model_orig.to_json()
        with open(filename_1, "w") as json_file:
            json_file.write(model_json)
        # serialize weights to HDF5
        model_orig.save_weights(filename_2)
        print("Saved model to disk for cross validation step :", fold-1)

        # later...
        '''
        # load json and create model
        json_file = open('model.json', 'r')
        loaded_model_json = json_file.read()
        json_file.close()
        loaded_model = model_from_json(loaded_model_json)
        # load weights into new model
        loaded_model.load_weights("model.h5")
        print("Loaded model from disk")
        '''
        # some time later...
        '''
        # load the model from disk
        loaded_model = pickle.load(open(filename, 'rb'))
        result = loaded_model.score(X_test, Y_test)
        print(result)
        '''
        # scores.append(best_svr.score(X_test, y_test))
        '''
        plt.figure()
        plt.plot(Y_test_fold, 'b')
        plt.plot(Y_pred_fold, 'r')
        '''
        # print('Accuracy =',accuracy_score(Y_test, Y_pred))
        Y_pred_fold = scaler_y.inverse_transform(Y_pred_fold)
        Y_test_fold = scaler_y.inverse_transform(Y_test_fold)
        r2_scores.append(r2_score(Y_test_fold, Y_pred_fold))
        mse_scores.append(mean_squared_error(Y_test_fold, Y_pred_fold))
        mape_runtime_scores.append(absolute_percentage_error(Y_test_fold[:,0], Y_pred_fold[:,0]))
        mape_power_scores.append(absolute_percentage_error(Y_test_fold[:,1], Y_pred_fold[:,1]))
        mae_scores.append(mean_absolute_error(Y_test_fold, Y_pred_fold))

    df = df.append({'model_name': 'dnn', 'dataset_name': dataset_name
                        , 'r2': r2_scores, 'mse': mse_scores, 'mape_runtime': mape_runtime_scores,'mape_power': mape_power_scores, 'mae': mae_scores }, ignore_index=True)
    k = k + 1  
    print(df.head())
    df.to_csv('\\DNN_results\\' + dataset_name + '.csv')


In [ ]:
dataset_name = 'dijkstra_physical'
print('Running Model for Dataset:', dataset_name)
dataset_path = '\\ALL_CSV\\Dataset\\' + dataset_name + '.csv'
path_for_saving_data = '\\Saved_Models_Data\\' + dataset_name
process_all(dataset_path, dataset_name, path_for_saving_data)

In [ ]:
dataset_name = 'matmul_physical'
print('Running Model for Dataset:', dataset_name)
dataset_path = '\\ALL_CSV\\Dataset\\' + dataset_name + '.csv'
path_for_saving_data = '\\Saved_Models_Data\\' + dataset_name
process_all(dataset_path, dataset_name, path_for_saving_data)

In [ ]:
dataset_name = 'matmul_simulated'
print('Running Model for Dataset:', dataset_name)
dataset_path = '\\ALL_CSV\\Dataset\\' + dataset_name + '.csv'
path_for_saving_data = '\\Saved_Models_Data\\' + dataset_name
process_all(dataset_path, dataset_name, path_for_saving_data)

In [34]:
dataset_name = 'dijkstra_simulated'
print('Running Model for Dataset:', dataset_name)
dataset_path = '\\ALL_CSV\\Dataset\\' + dataset_name + '.csv'
path_for_saving_data = '\\Saved_Models_Data\\' + dataset_name
process_all(dataset_path, dataset_name, path_for_saving_data)

Running Model for Dataset: dijkstra_simulated
Data X and Y shape (362, 22) (362, 2)
Train Test Split: (289, 22) (73, 22) (289, 2) (73, 2)
####################################################################
Saved model to disk for cross validation step : 1
Saved model to disk for cross validation step : 2
Saved model to disk for cross validation step : 3
Saved model to disk for cross validation step : 4
Saved model to disk for cross validation step : 5
Saved model to disk for cross validation step : 6
Saved model to disk for cross validation step : 7
Saved model to disk for cross validation step : 8
Saved model to disk for cross validation step : 9
Saved model to disk for cross validation step : 10
  model_name        dataset_name  \
0        dnn  dijkstra_simulated   

                                                  r2  \
0  [0.9751907413307365, 0.9883722336340972, 0.973...   

                                                 mse  \
0  [5538537.74640093, 10006822.53119802, 16539826.

In [7]:
dataset_name = 'svm_simulated'
print('Running Model for Dataset:', dataset_name)
dataset_path = '\\ALL_CSV\\Dataset\\' + dataset_name + '.csv'
path_for_saving_data = '\\Saved_Models_Data\\' + dataset_name
process_all(dataset_path, dataset_name, path_for_saving_data)

Running Model for Dataset: svm_simulated
Data X and Y shape (390, 22) (390, 2)
Train Test Split: (312, 22) (78, 22) (312, 2) (78, 2)
####################################################################
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Saved model to disk for cross validation step : 1
Saved model to disk for cross validation step : 2
Saved model to disk for cross validation step : 3
Saved model to disk for cross validation step : 4
Saved model to disk for cross validation step : 5
Saved model to disk for cross validation step : 6
Saved model to disk for cross validation step : 7
Saved model to disk for cross validation step : 8
Saved model to disk for cross validation step : 9
Saved model to disk for cross validation step : 10
  model_name   dataset_name  \
0        dnn  svm_simulated   

                                                  r2  \
0  [0.9904417855186105, 0.989701078778101, 0.9871...   

 

In [8]:
dataset_name = 'svm_physical'
print('Running Model for Dataset:', dataset_name)
dataset_path = '\\ALL_CSV\\Dataset\\' + dataset_name + '.csv'
path_for_saving_data = '\\Saved_Models_Data\\' + dataset_name
process_all(dataset_path, dataset_name, path_for_saving_data)

Running Model for Dataset: svm_physical
Data X and Y shape (52, 26) (52, 2)
Train Test Split: (41, 26) (11, 26) (41, 2) (11, 2)
####################################################################
Saved model to disk for cross validation step : 1
Saved model to disk for cross validation step : 2
Saved model to disk for cross validation step : 3
Saved model to disk for cross validation step : 4
Saved model to disk for cross validation step : 5
Saved model to disk for cross validation step : 6
Saved model to disk for cross validation step : 7
Saved model to disk for cross validation step : 8
Saved model to disk for cross validation step : 9
Saved model to disk for cross validation step : 10
  model_name  dataset_name                                                 r2  \
0        dnn  svm_physical  [0.4720475509611476, 0.8579091271970247, 0.187...   

                                                 mse  \
0  [5415236.214816459, 1722881.4519262502, 429994...   

                          

In [9]:
dataset_name = 'tracking_physical'
print('Running Model for Dataset:', dataset_name)
dataset_path = '\\ALL_CSV\\Dataset\\' + dataset_name + '.csv'
path_for_saving_data = '\\Saved_Models_Data\\' + dataset_name
process_all(dataset_path, dataset_name, path_for_saving_data)

Running Model for Dataset: tracking_physical
Data X and Y shape (52, 26) (52, 2)
Train Test Split: (41, 26) (11, 26) (41, 2) (11, 2)
####################################################################
Saved model to disk for cross validation step : 1
Saved model to disk for cross validation step : 2
Saved model to disk for cross validation step : 3
Saved model to disk for cross validation step : 4
Saved model to disk for cross validation step : 5
Saved model to disk for cross validation step : 6
Saved model to disk for cross validation step : 7
Saved model to disk for cross validation step : 8
Saved model to disk for cross validation step : 9
Saved model to disk for cross validation step : 10
  model_name       dataset_name  \
0        dnn  tracking_physical   

                                                  r2  \
0  [0.786876737096216, 0.7556139958185053, 0.7130...   

                                                 mse  \
0  [620189.5826894549, 1859593.686386227, 2211780...   



In [10]:
dataset_name = 'tracking_simulated'
print('Running Model for Dataset:', dataset_name)
dataset_path = '\\ALL_CSV\\Dataset\\' + dataset_name + '.csv'
path_for_saving_data = '\\Saved_Models_Data\\' + dataset_name
process_all(dataset_path, dataset_name, path_for_saving_data)

Running Model for Dataset: tracking_simulated
Data X and Y shape (425, 22) (425, 2)
Train Test Split: (340, 22) (85, 22) (340, 2) (85, 2)
####################################################################
Saved model to disk for cross validation step : 1
Saved model to disk for cross validation step : 2
Saved model to disk for cross validation step : 3
Saved model to disk for cross validation step : 4
Saved model to disk for cross validation step : 5
Saved model to disk for cross validation step : 6
Saved model to disk for cross validation step : 7
Saved model to disk for cross validation step : 8
Saved model to disk for cross validation step : 9
Saved model to disk for cross validation step : 10
  model_name        dataset_name  \
0        dnn  tracking_simulated   

                                                  r2  \
0  [0.9909958254606885, 0.9865979223988, 0.980709...   

                                                 mse  \
0  [68890.39565567755, 321425.16922584985, 393436.

In [11]:
dataset_name = 'stitch_physical'
print('Running Model for Dataset:', dataset_name)
dataset_path = '\\ALL_CSV\\Dataset\\' + dataset_name + '.csv'
path_for_saving_data = '\\Saved_Models_Data\\' + dataset_name
process_all(dataset_path, dataset_name, path_for_saving_data)

Running Model for Dataset: stitch_physical
Data X and Y shape (52, 26) (52, 2)
Train Test Split: (41, 26) (11, 26) (41, 2) (11, 2)
####################################################################
Saved model to disk for cross validation step : 1
Saved model to disk for cross validation step : 2
Saved model to disk for cross validation step : 3
Saved model to disk for cross validation step : 4
Saved model to disk for cross validation step : 5
Saved model to disk for cross validation step : 6
Saved model to disk for cross validation step : 7
Saved model to disk for cross validation step : 8
Saved model to disk for cross validation step : 9
Saved model to disk for cross validation step : 10
  model_name     dataset_name  \
0        dnn  stitch_physical   

                                                  r2  \
0  [0.7100145407956665, 0.9152769781240822, 0.811...   

                                                 mse  \
0  [2890886.376534916, 962445.2868391402, 1558760...   

      

In [14]:
dataset_name = 'stitch_simulated'
print('Running Model for Dataset:', dataset_name)
dataset_path = '\\ALL_CSV\\Dataset\\' + dataset_name + '.csv'
path_for_saving_data = '\\Saved_Models_Data\\' + dataset_name
process_all(dataset_path, dataset_name, path_for_saving_data)

Running Model for Dataset: stitch_simulated
Data X and Y shape (425, 22) (425, 2)
Train Test Split: (340, 22) (85, 22) (340, 2) (85, 2)
####################################################################
Saved model to disk for cross validation step : 1
Saved model to disk for cross validation step : 2
Saved model to disk for cross validation step : 3
Saved model to disk for cross validation step : 4
Saved model to disk for cross validation step : 5
Saved model to disk for cross validation step : 6
Saved model to disk for cross validation step : 7
Saved model to disk for cross validation step : 8
Saved model to disk for cross validation step : 9
Saved model to disk for cross validation step : 10
  model_name      dataset_name  \
0        dnn  stitch_simulated   

                                                  r2  \
0  [0.9923456179166921, 0.984609171546237, 0.9917...   

                                                 mse  \
0  [276794.8452023225, 259301.07284741732, 592022...   


In [15]:
dataset_name = 'mser_physical'
print('Running Model for Dataset:', dataset_name)
dataset_path = '\\ALL_CSV\\Dataset\\' + dataset_name + '.csv'
path_for_saving_data = '\\Saved_Models_Data\\' + dataset_name
process_all(dataset_path, dataset_name, path_for_saving_data)

Running Model for Dataset: mser_physical
Data X and Y shape (52, 26) (52, 2)
Train Test Split: (41, 26) (11, 26) (41, 2) (11, 2)
####################################################################
Saved model to disk for cross validation step : 1
Saved model to disk for cross validation step : 2
Saved model to disk for cross validation step : 3
Saved model to disk for cross validation step : 4
Saved model to disk for cross validation step : 5
Saved model to disk for cross validation step : 6
Saved model to disk for cross validation step : 7
Saved model to disk for cross validation step : 8
Saved model to disk for cross validation step : 9
Saved model to disk for cross validation step : 10
  model_name   dataset_name  \
0        dnn  mser_physical   

                                                  r2  \
0  [0.7629031285410384, 0.7275281193130377, 0.740...   

                                                 mse  \
0  [130181.9864344397, 368161.81668669026, 416694...   

            

In [16]:
dataset_name = 'mser_simulated'
print('Running Model for Dataset:', dataset_name)
dataset_path = '\\ALL_CSV\\Dataset\\' + dataset_name + '.csv'
path_for_saving_data = '\\Saved_Models_Data\\' + dataset_name
process_all(dataset_path, dataset_name, path_for_saving_data)

Running Model for Dataset: mser_simulated
Data X and Y shape (430, 22) (430, 2)
Train Test Split: (344, 22) (86, 22) (344, 2) (86, 2)
####################################################################
Saved model to disk for cross validation step : 1
Saved model to disk for cross validation step : 2
Saved model to disk for cross validation step : 3
Saved model to disk for cross validation step : 4
Saved model to disk for cross validation step : 5
Saved model to disk for cross validation step : 6
Saved model to disk for cross validation step : 7
Saved model to disk for cross validation step : 8
Saved model to disk for cross validation step : 9
Saved model to disk for cross validation step : 10
  model_name    dataset_name  \
0        dnn  mser_simulated   

                                                  r2  \
0  [0.9947005062750076, 0.9796324165524922, 0.985...   

                                                 mse  \
0  [14520.628286588488, 83645.40846077373, 91207....   

     

In [17]:
dataset_name = 'montecarlocalcpi_physical'
print('Running Model for Dataset:', dataset_name)
dataset_path = '\\ALL_CSV\\Dataset\\' + dataset_name + '.csv'
path_for_saving_data = '\\Saved_Models_Data\\' + dataset_name
process_all(dataset_path, dataset_name, path_for_saving_data)

Running Model for Dataset: montecarlocalcpi_physical
Data X and Y shape (260, 22) (260, 2)
Train Test Split: (208, 22) (52, 22) (208, 2) (52, 2)
####################################################################
Saved model to disk for cross validation step : 1
Saved model to disk for cross validation step : 2
Saved model to disk for cross validation step : 3
Saved model to disk for cross validation step : 4
Saved model to disk for cross validation step : 5
Saved model to disk for cross validation step : 6
Saved model to disk for cross validation step : 7
Saved model to disk for cross validation step : 8
Saved model to disk for cross validation step : 9
Saved model to disk for cross validation step : 10
  model_name               dataset_name  \
0        dnn  montecarlocalcpi_physical   

                                                  r2  \
0  [0.9234896656953197, 0.9025932743322768, 0.913...   

                                                 mse  \
0  [319819281584.06854, 38445

In [18]:
dataset_name = 'montecarlocalcpi_simulated'
print('Running Model for Dataset:', dataset_name)
dataset_path = '\\ALL_CSV\\Dataset\\' + dataset_name + '.csv'
path_for_saving_data = '\\Saved_Models_Data\\' + dataset_name
process_all(dataset_path, dataset_name, path_for_saving_data)

Running Model for Dataset: montecarlocalcpi_simulated
Data X and Y shape (1365, 23) (1365, 2)
Train Test Split: (1092, 23) (273, 23) (1092, 2) (273, 2)
####################################################################
Saved model to disk for cross validation step : 1
Saved model to disk for cross validation step : 2
Saved model to disk for cross validation step : 3
Saved model to disk for cross validation step : 4
Saved model to disk for cross validation step : 5
Saved model to disk for cross validation step : 6
Saved model to disk for cross validation step : 7
Saved model to disk for cross validation step : 8
Saved model to disk for cross validation step : 9
Saved model to disk for cross validation step : 10
  model_name                dataset_name  \
0        dnn  montecarlocalcpi_simulated   

                                                  r2  \
0  [0.9958960287706026, 0.9940884983718232, 0.994...   

                                                 mse  \
0  [1696842.52584298

In [19]:
dataset_name = 'qsort_physical'
print('Running Model for Dataset:', dataset_name)
dataset_path = '\\ALL_CSV\\Dataset\\' + dataset_name + '.csv'
path_for_saving_data = '\\Saved_Models_Data\\' + dataset_name
process_all(dataset_path, dataset_name, path_for_saving_data)

Running Model for Dataset: qsort_physical
Data X and Y shape (672, 22) (672, 2)
Train Test Split: (537, 22) (135, 22) (537, 2) (135, 2)
####################################################################
Saved model to disk for cross validation step : 1
Saved model to disk for cross validation step : 2
Saved model to disk for cross validation step : 3
Saved model to disk for cross validation step : 4
Saved model to disk for cross validation step : 5
Saved model to disk for cross validation step : 6
Saved model to disk for cross validation step : 7
Saved model to disk for cross validation step : 8
Saved model to disk for cross validation step : 9
Saved model to disk for cross validation step : 10
  model_name    dataset_name  \
0        dnn  qsort_physical   

                                                  r2  \
0  [0.8955967412352415, 0.9288838511515438, 0.870...   

                                                 mse  \
0  [1594720.9698034774, 438937.28761789633, 19910...   

   

In [20]:
dataset_name = 'qsort_simulated'
print('Running Model for Dataset:', dataset_name)
dataset_path = '\\ALL_CSV\\Dataset\\' + dataset_name + '.csv'
path_for_saving_data = '\\Saved_Models_Data\\' + dataset_name
process_all(dataset_path, dataset_name, path_for_saving_data)

Running Model for Dataset: qsort_simulated
Data X and Y shape (2730, 23) (2730, 2)
Train Test Split: (2184, 23) (546, 23) (2184, 2) (546, 2)
####################################################################
Saved model to disk for cross validation step : 1
Saved model to disk for cross validation step : 2
Saved model to disk for cross validation step : 3
Saved model to disk for cross validation step : 4
Saved model to disk for cross validation step : 5
Saved model to disk for cross validation step : 6
Saved model to disk for cross validation step : 7
Saved model to disk for cross validation step : 8
Saved model to disk for cross validation step : 9
Saved model to disk for cross validation step : 10
  model_name     dataset_name  \
0        dnn  qsort_simulated   

                                                  r2  \
0  [0.998134145269316, 0.9973872554165604, 0.9969...   

                                                 mse  \
0  [2236.942092665856, 2296.295906163151, 1344.68... 

In [21]:
dataset_name = 'sha_simulated'
print('Running Model for Dataset:', dataset_name)
dataset_path = '\\ALL_CSV\\Dataset\\' + dataset_name + '.csv'
path_for_saving_data = '\\Saved_Models_Data\\' + dataset_name
process_all(dataset_path, dataset_name, path_for_saving_data)

Running Model for Dataset: sha_simulated
Data X and Y shape (367, 22) (367, 2)
Train Test Split: (293, 22) (74, 22) (293, 2) (74, 2)
####################################################################
Saved model to disk for cross validation step : 1
Saved model to disk for cross validation step : 2
Saved model to disk for cross validation step : 3
Saved model to disk for cross validation step : 4
Saved model to disk for cross validation step : 5
Saved model to disk for cross validation step : 6
Saved model to disk for cross validation step : 7
Saved model to disk for cross validation step : 8
Saved model to disk for cross validation step : 9
Saved model to disk for cross validation step : 10
  model_name   dataset_name  \
0        dnn  sha_simulated   

                                                  r2  \
0  [0.9681358500612449, 0.9918126562924627, 0.983...   

                                                 mse  \
0  [859872.2632164496, 288059.6605304792, 1066618...   

        

In [22]:
dataset_name = 'sha_physical'
print('Running Model for Dataset:', dataset_name)
dataset_path = '\\ALL_CSV\\Dataset\\' + dataset_name + '.csv'
path_for_saving_data = '\\Saved_Models_Data\\' + dataset_name
process_all(dataset_path, dataset_name, path_for_saving_data)

Running Model for Dataset: sha_physical
Data X and Y shape (52, 21) (52, 2)
Train Test Split: (41, 21) (11, 21) (41, 2) (11, 2)
####################################################################
Saved model to disk for cross validation step : 1
Saved model to disk for cross validation step : 2
Saved model to disk for cross validation step : 3
Saved model to disk for cross validation step : 4
Saved model to disk for cross validation step : 5
Saved model to disk for cross validation step : 6
Saved model to disk for cross validation step : 7
Saved model to disk for cross validation step : 8
Saved model to disk for cross validation step : 9
Saved model to disk for cross validation step : 10
  model_name  dataset_name                                                 r2  \
0        dnn  sha_physical  [0.8841573722968434, 0.8036197675155613, 0.800...   

                                                 mse  \
0  [2343448.255951007, 6583634.1713729, 5961103.3...   

                          